In [1]:
from hcipy import *
from stgp_util_32 import *
import h5py

import numpy as np
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt

import scipy.ndimage as ndimage
import scipy
import time
import os

folder_name = '16x16_sn4'

filepath = 'matrix_for_GP_16x16.mat'
arrays = {}
f = h5py.File(filepath,'r')
for k, v in f.items():
    arrays[k] = np.array(v)

# the full phase matrix
mask_phase8 = arrays["mask_phase8"]

M2P = arrays["M2P"]
M2P = M2P.transpose().astype(np.float32)



wavelength_wfs = 0.5e-6 

G8 = arrays["G_8"]
G8 = G8.transpose().astype(np.float32)
G_recon = G8
SH_grid8 = get_non_empty_column_indices(G8)
G8 = G8[:,SH_grid8]

n_history = 5
A8          = scipy.linalg.block_diag(*([G8] * n_history))
A8          = np.concatenate((A8, np.zeros((A8.shape[0], 1 * G_recon.shape[1]))),1).astype(np.float32)

act_pix = G_recon.shape[1]
n_pixels = 16*8
mask_phase8 = arrays["mask_phase8"]

(xvalid, yvalid) = np.nonzero(mask_phase8)

vec_to_im    = vec_to_img(n_pixels, xvalid, yvalid)
im_to_vec    = img_to_vec(xvalid, yvalid)

filepath = 'dm2.mat'
arrays = {}
f = h5py.File(filepath,'r')
for k, v in f.items():
    arrays[k] = np.array(v)

M2P_dm = arrays["M2P_dm"]
M2P_dm = M2P_dm.transpose().astype(np.float32)

In [2]:
Cff_np = np.load(os.path.join(folder_name, 'Cff_np8.npy'))
Cwaff_np = np.load(os.path.join(folder_name, 'Cwaff_np8.npy'))

Cs_np = np.load(os.path.join(folder_name, 'Cs_np.npy'))

In [3]:
var_ff_img   = vec_to_im(np.diagonal(Cff_np))
var_waff_img = vec_to_im(np.diagonal(Cwaff_np))
var_s_img    = vec_to_im(np.diagonal(Cs_np))

In [4]:
constant = ((wavelength_wfs / (2 * np.pi)) * 1e9)**2

ff_max = np.amax(var_ff_img[:,:]*constant)
ff_min = np.amin(var_ff_img[:,:]*constant, where=var_waff_img[:,:]!=0, initial=2)
waff_max = np.amax(var_waff_img[:,:]*constant)
waff_min = np.amin(var_waff_img[:,:]*constant, where=var_waff_img[:,:]!=0, initial=2)
s_max = np.amax(var_s_img[:,:]*constant)
s_min = np.amin(var_s_img[:,:]*constant, where=var_s_img[:,:]!=0, initial=2)

fig, axs = plt.subplots(1,3,figsize = (10.4,2.3))
im1 = axs[0].imshow(var_ff_img[:,:]*constant, vmin=ff_min- 0.1*abs(ff_min-ff_max), vmax=ff_max)
plt.colorbar(im1, ax=axs[0], format='%.1e')
axs[0].axis(False)
axs[0].set_title('FF-GP variance')

im3 = axs[1].imshow(var_waff_img[:,:]*constant, vmin=waff_min- 0.1*(abs(waff_min-waff_max)), vmax=waff_max)
plt.colorbar(im3, ax=axs[1], format='%.1e')
axs[1].axis(False)
axs[1].set_title('WAFF-GP variance')

im2 = axs[2].imshow(var_s_img[:,:]*constant, vmin=s_min- 0.1*(abs(s_min-s_max)), vmax=s_max)
plt.colorbar(im2, ax=axs[2], format='%.1e')
axs[2].axis(False)
axs[2].set_title('S-GP variance')

plt.show()



<IPython.core.display.Javascript object>

In [5]:
#number of modes
n_modes = 224  
P2M = np.linalg.pinv(M2P[:,0:n_modes])
F = M2P[:,:n_modes] @ P2M 

P2M_dm = np.linalg.pinv(M2P_dm[:,:])
F = M2P_dm[:,:] @ P2M_dm 

modal_var_ff = F @ Cff_np @ F.transpose()
modal_var_waff = F @ Cwaff_np @ F.transpose()
modal_var_s = F @ Cs_np @ F.transpose()

var_ff_img_dm = vec_to_im(np.diagonal(modal_var_ff))
var_waff_img_dm = vec_to_im(np.diagonal(modal_var_waff))
var_s_img_dm  = vec_to_im(np.diagonal(modal_var_s))

In [6]:
constant = ((wavelength_wfs / (2 * np.pi)) * 1e9)


ff_max = np.amax(var_ff_img[:,:]*constant)
ff_min = np.amin(var_ff_img[:,:]*constant, where=var_waff_img[:,:]!=0, initial=2)
waff_max = np.amax(var_waff_img[:,:]*constant)
waff_min = np.amin(var_waff_img[:,:]*constant, where=var_waff_img[:,:]!=0, initial=2)
s_max = np.amax(var_s_img[:,:]*constant)
s_min = np.amin(var_s_img[:,:]*constant, where=var_s_img[:,:]!=0, initial=2)

fig, axs = plt.subplots(2,3,figsize = (10.4,5.3))
im1 = axs[0][0].imshow(var_ff_img[:,:]*constant, vmin=ff_min- 0.1*abs(ff_min-ff_max), vmax=ff_max)
plt.colorbar(im1, ax=axs[0][0], format='%.1e')
axs[0][0].axis(False)
axs[0][0].set_title('FF-GP')

im3 = axs[0][1].imshow(var_waff_img[:,:]*constant, vmin=waff_min- 0.1*(abs(waff_min-waff_max)), vmax=waff_max)
plt.colorbar(im3, ax=axs[0][1], format='%.1e')
axs[0][1].axis(False)
axs[0][1].set_title('WAFF-GP')

im2 = axs[0][2].imshow(var_s_img[:,:]*constant, vmin=s_min- 0.1*(abs(s_min-s_max)), vmax=s_max)
plt.colorbar(im2, ax=axs[0][2], format='%.1e')
axs[0][2].axis(False)
axs[0][2].set_title('S-GP')


ff_max = np.amax(var_ff_img_dm[:,:]*constant)
ff_min = np.amin(var_ff_img_dm[:,:]*constant, where=var_waff_img_dm[:,:]!=0, initial=2)
waff_max = np.amax(var_waff_img_dm[:,:]*constant)
waff_min = np.amin(var_waff_img_dm[:,:]*constant, where=var_waff_img_dm[:,:]!=0, initial=2)
s_max = np.amax(var_s_img_dm[:,:]*constant)
s_min = np.amin(var_s_img_dm[:,:]*constant, where=var_s_img_dm[:,:]!=0, initial=2)

im1 = axs[1][0].imshow(var_ff_img_dm[:,:]*constant, vmin=ff_min- 0.1*abs(ff_min-ff_max), vmax=ff_max)
plt.colorbar(im1, ax=axs[1][0], format='%.1e')
axs[1][0].axis(False)
axs[1][0].set_title('FF-GP, DM space')

im3 = axs[1][1].imshow(var_waff_img_dm[:,:]*constant, vmin=waff_min- 0.1*(abs(waff_min-waff_max)), vmax=waff_max)
plt.colorbar(im3, ax=axs[1][1], format='%.1e')
axs[1][1].axis(False)
axs[1][1].set_title('WAFF-GP, DM space')

im2 = axs[1][2].imshow(var_s_img_dm[:,:]*constant, vmin=s_min- 0.1*(abs(s_min-s_max)), vmax=s_max)
plt.colorbar(im2, ax=axs[1][2], format='%.1e')
axs[1][2].axis(False)
axs[1][2].set_title('S-GP, DM space')

plt.savefig('Uncertainty_highnoise.pdf')

plt.show()

<IPython.core.display.Javascript object>

In [7]:
fig, axs = plt.subplots(1,2,figsize = (7,2.3))

gain_ff = np.divide(var_s_img[:,:], var_ff_img[:,:], out=np.zeros_like(var_s_img[:,:]), where=var_ff_img[:,:]!=0)
gain_waff = np.divide(var_s_img[:,:], var_waff_img[:,:], out=np.zeros_like(var_s_img[:,:]), where=var_waff_img[:,:]!=0)

gain_max = np.amax(gain_ff)
gain_min = np.amin(gain_ff, where=var_ff_img[:,:]!=0, initial=10)
gain_max2 = np.amax(gain_waff)
gain_min2 = np.amin(gain_waff, where=var_waff_img[:,:]!=0, initial=10)

im4 = axs[0].imshow(gain_ff, vmin=gain_min - 0.1*abs(gain_min-gain_max), vmax=gain_max)
plt.colorbar(im4, ax=axs[0])
axs[0].axis(False)
axs[0].set_title('S-GP / FF-GP ')

im4 = axs[1].imshow(gain_waff, vmin=gain_min2 - 0.1*abs(gain_min2-gain_max2), vmax=gain_max2)
plt.colorbar(im4, ax=axs[1])
axs[1].axis(False)
axs[1].set_title('S-GP / WAFF-GP')
plt.show()

<IPython.core.display.Javascript object>

In [8]:
fig, axs = plt.subplots(1,2,figsize = (7,2.3))

gain_ff = np.divide(var_s_img_dm[:,:], var_ff_img_dm[:,:], out=np.zeros_like(var_s_img_dm[:,:]), where=var_ff_img[:,:]!=0)
gain_waff = np.divide(var_s_img_dm[:,:], var_waff_img_dm[:,:], out=np.zeros_like(var_s_img_dm[:,:]), where=var_waff_img[:,:]!=0)


gain_max = np.amax(gain_ff)
gain_min = np.amin(gain_ff, where=var_ff_img[:,:]!=0, initial=10)
gain_max2 = np.amax(gain_waff)
gain_min2 = np.amin(gain_waff, where=var_waff_img[:,:]!=0, initial=10)

im4 = axs[0].imshow(gain_ff, vmin=gain_min - 0.1*abs(gain_min-gain_max), vmax=gain_max)
plt.colorbar(im4, ax=axs[0])
axs[0].axis(False)
axs[0].set_title('S-GP / FF-GP ')

im4 = axs[1].imshow(gain_waff, vmin=gain_min2 - 0.1*abs(gain_min2-gain_max2), vmax=gain_max2)
plt.colorbar(im4, ax=axs[1])
axs[1].axis(False)
axs[1].set_title('S-GP / WAFF-GP')

plt.savefig('gain_highnoise.pdf')
plt.show()

<IPython.core.display.Javascript object>